## Univariate MLP Models

Multilayer Perceptrons, or MLPs for short, can be used to model univariate time series forecasting
problems. Univariate time series are a dataset comprised of a single series of observations with
a temporal ordering and a model is required to learn from the series of past observations to
predict the next value in the sequence. This section is divided into two parts; they are:

1. Data Preparation
2. MLP Model

### 1. Data Preparation

Before a univariate series can be modeled, it must be prepared. The MLP model will learn a
function that maps a sequence of past observations as input to an output observation. As such,
the sequence of observations must be transformed into multiple examples from which the model
can learn. Consider a given univariate sequence:

![Img_1](Imgs/sequence.png)

We can divide the sequence into multiple input/output patterns called samples, where three
time steps are used as input and one time step is used as output for the one-step prediction
that is being learned.

In [3]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence (Monterrey's Temperature in °C from Dec 1st - Dec 21st)
raw_seq = [21, 17, 17, 20, 18, 21, 19, 19, 22, 18,13, 12, 19, 23, 25, 25, 14, 9, 10, 13, 13]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[21 17 17] 20
[17 17 20] 18
[17 20 18] 21
[20 18 21] 19
[18 21 19] 19
[21 19 19] 22
[19 19 22] 18
[19 22 18] 13
[22 18 13] 12
[18 13 12] 19
[13 12 19] 23
[12 19 23] 25
[19 23 25] 25
[23 25 25] 14
[25 25 14] 9
[25 14  9] 10
[14  9 10] 13
[ 9 10 13] 13


Running the example above splits the univariate series into six samples where each sample has
three input time steps and one output time step.

Now that we know how to prepare a univariate series for modeling, let’s look at developing
an MLP model that can learn the mapping of inputs to outputs.

### 2. MLP Model

A simple MLP model has a single hidden layer of nodes, and an output layer used to make a
prediction. We can define an MLP for univariate time series forecasting as follows.


In [4]:
# univariate mlp example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

# define model

model = Sequential()
model.add(Dense(100, activation= 'relu' , input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer= 'adam' , loss= 'mse' )

Using TensorFlow backend.


Important in the definition is the shape of the input; that is what the model expects as
input for each sample in terms of the number of time steps. The number of time steps as input
is the number we chose when preparing our dataset as an argument to the `split sequence()`
function. The input dimension for each sample is specified in the input dim argument on the
definition of first hidden layer. Technically, the model will view each time step as a separate
feature instead of separate time steps.
We almost always have multiple samples, therefore, the model will expect the input
component of training data to have the dimensions or shape: ```[samples, features]```. Our
split sequence() function in the previous section outputs the X with the shape `[samples,features]` ready to use for modeling. The model is fit using the efficient Adam version of
stochastic gradient descent and optimized using the mean squared error, or ‘mse’, loss function.
Once the model is defined, we can fit it on the training dataset.

In [5]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

After the model is fit, we can use it to make a prediction. We can predict the next value
in the sequence by providing the input: `[70, 80, 90]`. And expecting the model to predict
something like: `[100]`. The model expects the input shape to be two-dimensional with `[samples,
features]`, therefore, we must reshape the single input sample before making the prediction,
e.g with the shape `[1, 3]` for 1 sample and 3 time steps used as input features.

In [8]:
# mean temperature of Dec 22 2019 prediction:
x_input = array([10, 13, 13])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)

print(yhat)

[[14.599053]]


We can tie all of this together and demonstrate how to develop an MLP for univariate time
series forecasting and make a single prediction.

In [22]:
# univariate mlp example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# define model
model = Sequential()
model.add(Dense(100, activation= 'relu' , input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer= 'adam' , loss= 'mse' )

# fit model
model.fit(X, y, epochs=2000, verbose=0)

# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)

print(yhat)

[[100.00855]]
